In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pickle
from sklearn.cross_validation import train_test_split
import itertools
import nltk
from nltk.corpus import stopwords
import unicodedata
import sys
%matplotlib inline 

/Users/Lucy/anaconda/envs/capstone/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [23]:
mypath = '/Users/Lucy/Google Drive/MSDS/2016Fall/DSGA1006/Data'
f = open(mypath + '/lstm-rnn/lstm_data.pickle','rb')
X_train, X_test, y_train, y_test = pickle.load(f)
f.close()

In [24]:
train_description = X_train.clean_description.copy()
test_description = X_test.clean_description.copy()
#train_description = [word for word in train_description if word not in stopwords.words('english')]
#test_description = [word for word in test_description if word not in stopwords.words('english')]

In [25]:
vocabulary_size = 5000
unknown_token = "UNKNOWNTOKEN"
sentence_start_token = "SENTENCESTART"
sentence_end_token = "SENTENCEEND"

In [26]:
# Split full descriptions into sentences
sentences = itertools.chain(*[[nltk.sent_tokenize(x.decode('utf-8').lower())] for x in train_description])
mod_sentences = []
for desc in sentences:
    full_desc = ""
    for x in desc:
        s = "%s %s %s " % (sentence_start_token, x, sentence_end_token)
        full_desc = full_desc + s
    mod_sentences.append(full_desc)

In [27]:
def strip_punctuation(text):
    """
    >>> strip_punctuation(u'something')
    u'something'

    >>> strip_punctuation(u'something.,:else really')
    u'somethingelse really'
    """
    punctutation_cats = set(['Pc', 'Pd', 'Ps', 'Pe', 'Pi', 'Pf', 'Po'])
    return ''.join(x for x in text
                   if unicodedata.category(x) not in punctutation_cats)

In [28]:
mod_sentences = map(strip_punctuation, mod_sentences)

In [29]:
for i in mod_sentences[:5]:
    print i + '\n'

SENTENCESTART cytori therapeutics inc is dedicated to developing and delivering innovative cellbased therapies to improve the quality and length of life SENTENCEEND SENTENCESTART we are a global public company engaged in the development and commercialization of stem and regenerative cell therapies for the treatment of cardiovascular disease reconstructive surgery and other serious chronic and life threatening conditions SENTENCEEND 

SENTENCESTART avva is a personal online application that helps women who have been diagnosed with breast cancer prepare for important medical appointments SENTENCEEND SENTENCESTART avvas vision is to empower and support patients by providing an online tool that serves as a comprehensive home base for breast cancer treatment  one that helps patients address the organizational emotional social and medical components of treatment SENTENCEEND SENTENCESTART avva is a personal advocate that gives patients tools to take a more active role in their own treatment S

In [30]:
tokenized_sentences = [nltk.word_tokenize(sent) for sent in mod_sentences]

In [35]:
for i in tokenized_sentences[3]:
    print i

SENTENCESTART
UNKNOWNTOKEN
is
a
company
that
brings
services
to
the
reader
based
on
the
context
of
the
page
they
are
on
SENTENCEEND
SENTENCESTART
we
make
the
page
more
rich
and
responsive
more
entertaining
SENTENCEEND
SENTENCESTART
combining
UNKNOWNTOKEN
with
a
search
engine
and
social
network
capabilities
we
change
the
web
page
from
a
UNKNOWNTOKEN
of
text
with
UNKNOWNTOKEN
to
a
dynamic
and
social
page
SENTENCEEND
SENTENCESTART
you
can
write
on
the
page
exchange
ideas
follow
ideas
or
discover
extra
information
brought
by
the
search
engine
just
by
UNKNOWNTOKEN
the
text
of
interest
SENTENCEEND
SENTENCESTART
we
bring
all
the
UNKNOWNTOKEN
of
what
the
internet
really
is
to
the
page
you
are
reading
SENTENCEEND
SENTENCESTART
UNKNOWNTOKEN
brings
improved
productivity
contextual
collaboration
and
intuitive
UNKNOWNTOKEN
capabilities
SENTENCEEND


In [37]:
y_train[3]

u'Internet'

In [32]:
# Count the word frequencies
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print "Found %d unique words tokens." % len(word_freq.items())

Found 74070 unique words tokens.


In [10]:
#word_freq = sorted(word_freq.items(), key=lambda i: i[1], reverse=True)

In [33]:
# Get the most common words and build index_to_word and word_to_index vectors
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

print "Using vocabulary size %d." % vocabulary_size
print "The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1])

# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]
    


Using vocabulary size 5000.
The least frequent word in our vocabulary is 'ar' and appeared 25 times.


In [34]:
{word_to_index.keys()[word_to_index.values().index(k)]: k for k in sorted(word_to_index.values())[:2]}

{u'SENTENCEEND': 1, u'and': 0}